# GPU-Jupyter

This Jupyterlab Instance is connected to the GPU via CUDA drivers. In this notebook, we test the installation and perform some basic operations on the GPU.

## Test GPU connection

#### Using the following command, your GPU type and its NVIDIA-SMI driver version should be listed:

In [1]:
!nvidia-smi

Wed Feb 17 14:38:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 207...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   43C    P0    35W / 215W |    378MiB /  7974MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Now, test if PyTorch can access the GPU via CUDA:

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(tf.config.list_physical_devices('XLA_GPU'))
device_lib.list_local_devices()

[PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13215508726155993711,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 139821540191525192
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6702805824
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1663900576452137187
 physical_device_desc: "device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7474961230302209906
 physical_device_desc: "device: XLA_GPU device"]

In [4]:
from __future__ import print_function
import numpy as np
import torch
a = torch.rand(5, 3)
a

tensor([[0.8067, 0.0610, 0.7669],
        [0.5577, 0.9805, 0.9937],
        [0.7162, 0.0951, 0.7936],
        [0.3258, 0.3903, 0.1614],
        [0.0854, 0.7302, 0.3586]])

## Performance test

#### Now we want to know how much faster a typical operation is using GPU. Therefore we do the same operation in numpy, PyTorch and PyTorch with CUDA. The test operation is the calculation of the prediction matrix that is done in a linear regression.

### 1) Numpy

In [5]:
x = np.random.rand(10000, 256)

In [6]:
%%timeit
H = x.dot(np.linalg.inv(x.transpose().dot(x))).dot(x.transpose())

399 ms ± 105 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2) PyTorch

In [7]:
x = torch.rand(10000, 256)

In [8]:
%%timeit
# Calculate the projection matrix of x on the CPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

85.5 ms ± 7.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 3) PyTorch on GPU via CUDA

In [9]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 256, device=device) # directly create a tensor on GPU
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])
    print(y.to("cpu", torch.double)[0:5, 0:5])

tensor([[0.2231, 0.7991, 0.9068, 0.1942, 0.6713],
        [0.5470, 0.8153, 0.9055, 0.4727, 0.9508],
        [0.5037, 0.5240, 0.1078, 0.8854, 0.4258],
        [0.6050, 0.3252, 0.0099, 0.6743, 0.6130],
        [0.1578, 0.0949, 0.4291, 0.6993, 0.4846]], device='cuda:0')
tensor([[0.2231, 0.7991, 0.9068, 0.1942, 0.6713],
        [0.5470, 0.8153, 0.9055, 0.4727, 0.9508],
        [0.5037, 0.5240, 0.1078, 0.8854, 0.4258],
        [0.6050, 0.3252, 0.0099, 0.6743, 0.6130],
        [0.1578, 0.0949, 0.4291, 0.6993, 0.4846]], dtype=torch.float64)


In [10]:
%%timeit
# Calculate the projection matrix of x on the GPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

11.4 ms ± 73.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Exhaustive Testing on GPU

In [11]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 10, device=device) # directly create a tensor on GPU

In [12]:
if torch.cuda.is_available():
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])

tensor([[0.2355, 0.1317, 0.2859, 0.8466, 0.1337],
        [0.3938, 0.7791, 0.2917, 0.4408, 0.7291],
        [0.1357, 0.5570, 0.7522, 0.7647, 0.8580],
        [0.8348, 0.0937, 0.5582, 0.5634, 0.0118],
        [0.4265, 0.1612, 0.1479, 0.9937, 0.3776]], device='cuda:0')


In [13]:
if torch.cuda.is_available():
    # Here is the memory of the GPU a border. 
    # A matrix with 100000 lines requires 37 GB, but only 8 GB are available.
    H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

In [14]:
if torch.cuda.is_available():
    print(H[0:5, 0:5])

tensor([[ 1.0220e-03, -9.5841e-06,  7.4560e-05,  6.4476e-04,  5.7171e-04],
        [-9.5841e-06,  1.1702e-03,  7.3497e-04,  2.9419e-04, -1.2504e-04],
        [ 7.4560e-05,  7.3497e-04,  1.2266e-03,  2.4695e-05, -1.0073e-04],
        [ 6.4476e-04,  2.9419e-04,  2.4695e-05,  1.2818e-03,  3.1767e-04],
        [ 5.7171e-04, -1.2504e-04, -1.0073e-04,  3.1767e-04,  1.1567e-03]],
       device='cuda:0')


In [15]:
if torch.cuda.is_available():
    # This operation is difficult, as an symmetric matrix is transferred 
    # back to the CPU. Is possible up to 30000 rows.
    print(H.to("cpu", torch.double)[0:5, 0:5])

tensor([[ 1.0220e-03, -9.5841e-06,  7.4560e-05,  6.4476e-04,  5.7171e-04],
        [-9.5841e-06,  1.1702e-03,  7.3497e-04,  2.9419e-04, -1.2504e-04],
        [ 7.4560e-05,  7.3497e-04,  1.2266e-03,  2.4695e-05, -1.0073e-04],
        [ 6.4476e-04,  2.9419e-04,  2.4695e-05,  1.2818e-03,  3.1767e-04],
        [ 5.7171e-04, -1.2504e-04, -1.0073e-04,  3.1767e-04,  1.1567e-03]],
       dtype=torch.float64)
